Download the catalogue from: https://dr7.lamost.org/catalogue \
Download `LAMOST LRS Stellar Parameter Catalog of A, F, G and K Stars`
![lamost catalog page](<images/lamost-catalog.png>)

In [1]:
import vaex

In [ ]:
file = "<downloaded file name>"

In [ ]:
# open with vaex
df = vaex.open(file)

In [ ]:
# extract the following columns
cols = ["subclass","teff", "teff_err", "logg", "logg_err", "feh", "feh_err", "rv", "rv_err", "alpha_m", "alpha_m_err", "gaia_source_id"]
df_filtered = df[cols]
df_filtered = df_filtered.extract()

In [ ]:
# change type from 'Byte' to 'str'
source_id = df_filtered["gaia_source_id"].to_numpy().astype(str)
subclass = df_filtered["subclass"].to_numpy().astype(str)

In [ ]:
# replace the columns

df_filtered.drop(["gaia_source_id", "subclass"], inplace=True)
df_filtered["source_id"] = source_id
df_filtered["subclass"] = subclass

In [ ]:
# filter -9999

masking = ((df_filtered["teff_err"].to_numpy() != -9999.0)*
           (df_filtered["logg_err"].to_numpy() != -9999.0)*
           (df_filtered["feh_err"].to_numpy() != -9999.0)*
           (df_filtered["rv_err"].to_numpy() != -9999.0)*
           (df_filtered["alpha_m_err"].to_numpy() != -9999.0))

In [ ]:
# convert to pandas dataframe to avoid stu*id "AttributeError: 'FitsBinTable' object has no attribute 'snake_name'"

df_pandas = df_filtered.to_pandas_df()

In [ ]:
# apply masking

df_clean = df_pandas[masking]

In [ ]:
# remove duplicates, if any
df_clean.drop_duplicates("source_id", inplace=True)
df = vaex.from_pandas(df_clean)

In [ ]:
# rename column names

cols_name = ["teff_lamost", "e_teff_lamost", "logg_lamost", "e_logg_lamost", "feh_lamost", "e_feh_lamost", "rv_lamost", 
             "e_rv_lamost", "alpham_lamost", "e_alpham_lamost", "source_id", "subclass_lamost"]
for old, new in zip(df.column_names, cols_name):
    df.rename(old, new,)

In [ ]:
# save as hdf5, if needed

df.export("lamost.hdf5", progress=True)